In [1]:
#-*- coding:utf-8 -*-

import os
import sys
import time
import random
import math
import pickle
import unicodedata

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import networkx as nx
import scipy.stats as st
import fathon
from fathon import fathonUtils as fu

from datetime import datetime, timedelta
from tqdm import tqdm
from tqdm.contrib.concurrent import process_map
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

from rpy2.robjects.packages import importr
from rpy2.robjects import robject

from statsmodels.tsa.api import VAR

import rpy2.robjects.numpy2ri
import rpy2.ipython.html
rpy2.robjects.numpy2ri.activate()
# rpy2.ipython.html.init_printing()

rTE = importr('RTransferEntropy', lib_loc="/home/yongkyung/R/x86_64-pc-linux-gnu-library/3.6")
rTE.set_quiet(True)

from utils_p import *

# setup seed
def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
#     torch.backends.cudnn.deterministic = True

SEED = 12345
seed_everything(SEED)

In [2]:
outcome_all = []
for lag in [10]:
    for noise in [2]:
        results_all = []
        for rep in tqdm(range(1)):
            n = 120
            while True:
                x = [100] * n + np.random.normal(0, 1, n)*noise

                for i in np.arange(10,n):
                    if i < 100:
                        x[i] = 0.95 * x[i-1] + np.random.normal(0,1)*noise
                    else: 
                        x[i] = 1.10 * x[i-1] + np.random.normal(0,1)*noise
                x = pd.Series(x)

                if np.min(x) > 0:
                    break

            k = 0
            while True:
                random.seed(k)
                np.random.seed(k)

                y = [70] * n + np.random.normal(0, 1, n)*noise
                for i in range(lag, n):
                    y[i] = 0.5 * x[i-lag] + 20 + np.random.normal(0,1)*noise
                y = pd.Series(y)

                out = []
                for lag_test in np.arange(1,25,1):
                    x_copy = x.to_numpy().copy()
                    y_copy = y.to_numpy().copy()

                    ETE_value = rTE.calc_ete(x = x_copy[1:(len(x_copy)-lag_test)+1], y = y_copy[(lag_test):len(y_copy)],lx=1,ly=1)
                    out.append([lag_test, np.asarray(ETE_value).item()])

                #check TE
                if lag == (np.argmax(np.array(out)[:,1])+1):
                    break

                k += 1    

            df = pd.DataFrame([x,y], index=['x', 'y']).T
            df_diff = np.log(df).diff(1).dropna()
            df_diff = df_diff.reset_index()

            out_all = []
            out_all.append(lag)
            out_all.append(noise)

            # pearson correlation
            corr_out = []
            for i in np.arange(1,25):
                corr_out.append([i, df_diff['x'].iloc[:-i].corr(df_diff['y'].iloc[i:])])
            corr_out = np.array(corr_out)
            top_lag = corr_out[np.argmax(corr_out[:,1])][0]
            out_all.append(top_lag)

            # rhoDCCA
            def get_rhoDCCA(x,y):
                x_f = fu.toAggregated(x)
                y_f = fu.toAggregated(y)

                pydcca = fathon.DCCA(x_f, y_f)
                winSizes = fu.linRangeByStep(4, 12, step=2)# n size
                polOrd = 1

                n, rho = pydcca.computeRho(winSizes, polOrd=polOrd)

                return pd.DataFrame(rho, index=n)
            
            corr_out = []
            for i in np.arange(1,25):
                rhoDCCA = get_rhoDCCA(df_diff['x'].iloc[:-i].to_numpy(),df_diff['y'].iloc[i:].to_numpy())
                rhoDCCA.columns = [i]
                corr_out.append(rhoDCCA)          
            corr_out_df = pd.concat(corr_out, axis=1)
            out_all = out_all + corr_out_df.T.idxmax().tolist()
            
            results_all.append(out_all)

        results_all = np.array(results_all)

        outcome = []
        outcome.append(lag)
        outcome.append(noise)
        for i in np.arange(2,7):
            outcome.append(mean_absolute_error(results_all[:,i], [lag]*len(results_all)))
            # outcome.append(mean_squared_error(results_all[:,i], [lag]*len(results_all), squared=False))
        print(outcome)
        outcome_all.append(outcome)        

100%|██████████| 1/1 [00:00<00:00,  1.77it/s]

[10, 2, 14.0, 0.0, 0.0, 9.0, 0.0]


In [3]:
outcome_all = []
for lag in [5, 10, 15]:
    for noise in [1, 2, 3]:
        results_all = []
        for rep in tqdm(range(100)):
            n = 120
            while True:
                x = [100] * n + np.random.normal(0, 1, n)*noise

                for i in np.arange(10,n):
                    if i < 100:
                        x[i] = 0.95 * x[i-1] + np.random.normal(0,1)*noise
                    else: 
                        x[i] = 1.10 * x[i-1] + np.random.normal(0,1)*noise
                x = pd.Series(x)

                if np.min(x) > 0:
                    break

            k = 0
            while True:
                random.seed(k)
                np.random.seed(k)

                y = [70] * n + np.random.normal(0, 1, n)*noise
                for i in range(lag, n):
                    y[i] = 0.5 * x[i-lag] + 20 + np.random.normal(0,1)*noise
                y = pd.Series(y)

                out = []
                for lag_test in np.arange(1,25,1):
                    x_copy = x.to_numpy().copy()
                    y_copy = y.to_numpy().copy()

                    ETE_value = rTE.calc_ete(x = x_copy[1:(len(x_copy)-lag_test)+1], y = y_copy[(lag_test):len(y_copy)],lx=1,ly=1)
                    out.append([lag_test, np.asarray(ETE_value).item()])

                #check TE
                if lag == (np.argmax(np.array(out)[:,1])+1):
                    break

                k += 1    

            df = pd.DataFrame([x,y], index=['x', 'y']).T
            df_diff = np.log(df).diff(1).dropna()
            df_diff = df_diff.reset_index()

            out_all = []
            out_all.append(lag)
            out_all.append(noise)

            # pearson correlation
            corr_out = []
            for i in np.arange(1,25):
                corr_out.append([i, df_diff['x'].iloc[:-i].corr(df_diff['y'].iloc[i:])])
            corr_out = np.array(corr_out)
            top_lag = corr_out[np.argmax(corr_out[:,1])][0]
            out_all.append(top_lag)

            # rhoDCCA
            def get_rhoDCCA(x,y):
                x_f = fu.toAggregated(x)
                y_f = fu.toAggregated(y)

                pydcca = fathon.DCCA(x_f, y_f)
                winSizes = fu.linRangeByStep(4, 12, step=2) # n size
                polOrd = 1

                n, rho = pydcca.computeRho(winSizes, polOrd=polOrd)

                return pd.DataFrame(rho, index=n)
            
            corr_out = []
            for i in np.arange(1,25):
                rhoDCCA = get_rhoDCCA(df_diff['x'].iloc[:-i].to_numpy(),df_diff['y'].iloc[i:].to_numpy())
                rhoDCCA.columns = [i]
                corr_out.append(rhoDCCA)          
            corr_out_df = pd.concat(corr_out, axis=1)
            out_all = out_all + corr_out_df.T.idxmax().tolist()
            
            results_all.append(out_all)

        results_all = np.array(results_all)

        outcome = []
        outcome.append(lag)
        outcome.append(noise)
        for i in np.arange(2,8):
            outcome.append(mean_absolute_error(results_all[:,i], [lag]*len(results_all)))
            # outcome.append(mean_squared_error(results_all[:,i], [lag]*len(results_all), squared=False))
        print(outcome)
        outcome_all.append(outcome)    

  0%|          | 0/100 [00:00<?, ?it/s]

[5, 1, 4.0, 0.12, 0.12, 0.12, 0.12, 0.12]


  0%|          | 0/100 [00:00<?, ?it/s]

[5, 2, 11.07, 4.95, 16.83, 0.0, 16.83, 0.0]


  0%|          | 0/100 [00:00<?, ?it/s]

...

[15, 1, 1.02, 7.92, 0.0, 0.0, 0.0, 0.0]


  0%|          | 0/100 [00:00<?, ?it/s]

[15, 2, 7.0, 7.0, 7.0, 7.0, 0.0, 0.0]


100%|██████████| 100/100 [00:54<00:00,  1.83it/s]

[15, 3, 7.0, 7.0, 7.0, 7.0, 0.0, 0.0]


In [4]:
pd.DataFrame(outcome_all)

,0,1,2,3,4,5,6,7
0,5,1,4.00,0.12,0.12,0.12,0.12,0.12
1,5,2,11.07,4.95,16.83,0.00,16.83,0.00
2,5,3,4.00,2.50,0.00,2.50,0.00,0.00
3,10,1,11.95,0.00,0.00,0.00,0.00,0.00
4,10,2,1.00,0.00,0.00,0.00,12.00,0.00
5,10,3,1.00,0.00,0.00,0.00,12.00,0.00
6,15,1,1.02,7.92,0.00,0.00,0.00,0.00
7,15,2,7.00,7.00,7.00,7.00,0.00,0.00
8,15,3,7.00,7.00,7.00,7.00,0.00,0.00


In [6]:
outcome_df = pd.DataFrame(outcome_all, columns=['lag', 'noise', 'TLCC', 'DCCA(4)', 'DCCA(6)', 'DCCA(8)', 'DCCA(10)', 'DCCA(12)']).set_index(['lag','noise']).T
outcome_df

lag         5                  10                15          
noise        1      2    3      1     2     3     1    2    3
TLCC      4.00  11.07  4.0  11.95   1.0   1.0  1.02  7.0  7.0
DCCA(4)   0.12   4.95  2.5   0.00   0.0   0.0  7.92  7.0  7.0
DCCA(6)   0.12  16.83  0.0   0.00   0.0   0.0  0.00  7.0  7.0
DCCA(8)   0.12   0.00  2.5   0.00   0.0   0.0  0.00  7.0  7.0
DCCA(10)  0.12  16.83  0.0   0.00  12.0  12.0  0.00  0.0  0.0
DCCA(12)  0.12   0.00  0.0   0.00   0.0   0.0  0.00  0.0  0.0

In [8]:
outcome_df.mean(axis=1)

TLCC        5.337778
DCCA(4)     3.276667
DCCA(6)     3.438889
DCCA(8)     1.846667
DCCA(10)    4.550000
DCCA(12)    0.013333
dtype: float64

In [9]:
outcome_df.to_csv('outcome.csv')